In [ ]:
# https://huggingface.co/blog/lcm_lora
import datetime
import os
import time
import warnings

# On Intel without a GPU, the cuda backend will spew errors and warnings.
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
warnings.filterwarnings("ignore")

from diffusers import DiffusionPipeline, LCMScheduler
import ipyplot
import torch
from IPython.display import display

pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", variant="fp16")
pipe.load_lora_weights("latent-consistency/lcm-lora-sdxl")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
if torch.backends.mps.is_available():
    pipe.to(device="mps", dtype=torch.float16)
    print("Using MPS")
elif torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True
    pipe.to(device="cuda", dtype=torch.float16)
    print("Using CUDA")
else:
    try:
        # Interestingly I see no performance improvement at all.
        import intel_extension_for_pytorch as ipex
        t = torch.float # bfloat16
        pipe.unet = ipex.optimize(pipe.unet.eval(), dtype=t, inplace=True)
        pipe.vae = ipex.optimize(pipe.vae.eval(), dtype=t, inplace=True)
        pipe.text_encoder = ipex.optimize(pipe.text_encoder.eval(), dtype=t, inplace=True)
        print("Using Intel (accelerated)")
    except ImportError:
        print("Using CPU (non-accelerated)")
        pass

def get_generator(seed):
    if torch.backends.mps.is_available():
        return torch.Generator("mps").manual_seed(seed)
    if torch.cuda.is_available():
        return torch.Generator("cuda").manual_seed(seed)
    generator = torch.Generator()
    generator.manual_seed(seed)
    return generator

In [ ]:
#prompt = "close-up photography of old man standing in the rain at night, in a street lit by lamps, leica 35mm summilux"
#prompt = "scene from the 1979 dystopian action film Mad Max, 35mm Eastmancolor"
prompt = "fireplace, warm cozy book shelf candle snow mountain window sofa rustic"
seed = 30

start = time.time()
img = pipe(prompt=prompt, num_inference_steps=4,
    guidance_scale=1, generator=get_generator(seed)).images[0]
print(f"Inference in {time.time()-start:.1f}s")
display(img)

In [ ]:
now = str(datetime.datetime.now().replace(microsecond=0)).replace(" ", "-").replace(":", "-")
img.save(f"out/sdxl_lcm_lora-{now}.png")